## Title: GLOBBIOMASS Global Biomass Data Products

### Description
These maps of forest growing stock volume and above-ground biomass are derived from Earth Observation data in the framework of ESA's GlobBiomass project.<br>

The data products consist of global datasets including estimates of<br>

- growing stock volume (GSV, unit: m3/ha) for the year 2010<br>

Definition: volume of all living trees more than 10 cm in diameter at breast height measured over bark from ground or stump height to a top stem diameter of 0 cm. Excludes: smaller branches, twigs, foliage, flowers, seeds, stump and roots (definition of FAO).<br>

A separate data layer is provided with per-pixel uncertainty expressed as standard error in m3/ha.<br>

- above ground biomass (AGB, unit: tons/ha i.e., Mg/ha) for the year 2010<br>

Definition: the mass, expressed as oven-dry weight of the woody parts (stem, bark, branches and twigs) of all living trees excluding stump and roots.<br>

A separate data layer is provided with per-pixel uncertainty expressed as standard error in Mg/ha.


### FLINT
This dataset has been checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: [Land Sector Datasets](https://datasets.mojaglobal.workers.dev/) https://datasets.mojaglobal.workers.dev/<br>

### Format
<b>Extent: </b>Global land coverage <br>
<b>Resolution:</b> Raster 40° x 40° tiles of the global GSV and AGB data - resolution 0.001 deg pixels<br>
<b>File type:</b> geotiff (.tif)<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Temporal Resolution: </b>2010<br>
<b>Size:</b> 46GB

### Original source
https://globbiomass.org/wp-content/uploads/GB_Maps/Globbiomass_global_dataset.html<br>
The full datasets (29 tiles) can be downloaded by using this Python script http://globbiomass.org/wp-content/uploads/GB_Maps/globbiomass_download.zip see code below to execute  Map on [this site](https://globbiomass.org/wp-content/uploads/GB_Maps/Globbiomass_global_dataset.html) can be used to download individual tiles.<br>
Global coverage(land), tiled 40° x 40° degrees tiles, geotiff (.tif)<br>
cell size ~0.000889 degree, resolution 45000 x 45000 Pixels<br>
Cordinate system EPSG: 4326 (WGS84)

### Licence
The maps are provided free of charge to encourage further research in the field of forest parameter retrieval and investigations that require spatially explicit information on forest parameters.

### Citation
Santoro, M., Cartus, O., Mermoz, S., Bouvet, A., Le Toan, T., Carvalhais, N., Rozendaal, D., Herold, M., Avitabile, V., Quegan, S., Carreiras, J., Rauste, Y., Balzter, H., Schmullius, C., Seifert, F.M., 2018, GlobBiomass global above-ground biomass and growing stock volume datasets, available on-line at http://globbiomass.org/products/global-mapping<br>
or<br>
Santoro, M. et al. (2018): GlobBiomass - global datasets of forest biomass. PANGAEA, https://doi.org/10.1594/PANGAEA.894711

### Metadata
Please make sure you read the technical specifications of the datasets to get acquainted with its features. The README file can be downloaded following this link: 
https://globbiomass.org/wp-content/uploads/GB_Maps/README_GLOBBIOMASS_global_20180531_web.pdf

Summary of GSV and AGB retrieval algorithms: The GSV estimates were obtained from spaceborne SAR (ALOS PALSAR, Envisat ASAR), optical (Landsat-7), LiDAR (ICESAT) and auxiliary datasets with multiple estimation procedures.
AGB was obtained from GSV with a set of Biomass Expansion and Conversion Factors (BCEF)
following approaches to extend on ground estimates of wood density and stem-to-total biomass
expansion factors to obtain a global raster dataset.
It is referred to the Algorithm Theoretical Basis Document for details on the EO datasets, the biomass
retrieval algorithms and the estimation of the BCEF
(see http://globbiomass.org/wpcontent/uploads/DOC/Deliverables/D6_D7/GlobBiomass_D6_7_Global_ATBD_final.pdf)

### Notes
NOTE: Users should consider that the maps do not provide exact values at the level of a single pixel (1 hectare) or a few pixels. Care needs to be taken in areas of strong topography because of the viewing effects on the Earth Observation data.

### Processing
Code is provided below to bulk download tiles for global coverage, or you can head to the original source to download select tiles. Code is also provided to mosaic the tiles into a single dataset, and resample the raster resolution.

In [ ]:
# The full datasets (29 tiles) can be downloaded by using this Python script http://globbiomass.org/wp-content/uploads/GB_Maps/globbiomass_download.zip 
#To download the global AGB and uncertainty maps (NOTE: 23 GB of data), download the script and run the following command
globbiomass_download.py -m AGB -o $output_path
#To download the global GSV and uncertainty maps (NOTE: 23 GB of data), download the script and run the following command
globbiomass_download.py -m GSV -o $output_path
                                                

### Unzip -python

In [ ]:
#Unzip all rasters
import zipfile,fnmatch,os
rootPath = r""
pattern = '*.zip'
for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        print(os.path.join(root, filename))
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(root, os.path.splitext(filename)[0]))

### Seamless mosaic -gdal

In [ ]:
#To create a seamless mosaic, e.g., for the AGB dataset, use the gdalbuildvrt command of the GDAL tools
gdalbuildvrt AGB.vrt */*agb.tif
#example
gdalbuildvrt -a_srs EPSG:4326 -r nearest -input_file_list C:/buildvrtInputFiles.txt C:/Data/AGB.vrt

### Resample or batch resample to change resolution -gdal

In [ ]:
#Use this code to resample a different raster resolution at -tr using GDAL (or QGIS)
gdalwarp -t_srs EPSG:4326 -tr 0.001 0.001 -r near -te -180.0 -90.0 180.0 90.0 -te_srs EPSG:4326 -of GTiff "C:/input.tif" "C:/resampledTile.tif"

In [ ]:
#Use this code in OSGEO4W Shell to batch processing of all tiles
for /R C:\GLOBBIOMASS\gsv\tifs %f IN (*.tif) do gdalwarp.exe -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9  -t_srs EPSG:4326 -tr 0.0005 0.0005 -r near -of GTiff %f C:\downloads\GLOBBIOMASS\gsv_TIF/%~nf.tif